In [1]:
import requests
import pandas as pd
from io import BytesIO
import os
from core.utils import solve_path, solve_dir, check_file_exists

In [2]:
ORIGINAL_DATA_DIR = solve_dir('original_data')
GENERATED_DATA_DIR = solve_dir('generated_data')
GENERATED_DATA_DIR = solve_path('estrutura_pdm', GENERATED_DATA_DIR)
GENERATED_DATA_DIR = solve_dir(GENERATED_DATA_DIR)

In [3]:
link_pdm_final = 'https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/governo/arquivos/pdm-2124/Planilha%20PdM%2021-24%20Dados%20Abertos.xlsx'

In [4]:
def download_xl(link, fname, folder, **read_excel):
    
    fpath = solve_path(fname, parent=folder)
    
    if os.path.exists(fpath):
        return pd.read_excel(fpath)
    
    with requests.get(link) as r:
        data = r.content
    data = BytesIO(data)
    df = pd.read_excel(data, **read_excel)
    df.to_excel(fpath, index=False)
    
    return df

In [5]:
df = download_xl(link_pdm_final, 'pdm_versao_final.xlsx',
                 ORIGINAL_DATA_DIR, skiprows=1)

In [6]:
de_para_metas = {
    'Meta_numero' : 'codigo',
    'Meta_descricao' : 'titulo',
    'Contexto' : 'contexto',
    'Informações Complementares' : 'complemento',
    'Eixo' : 'macrotema',
    'Objetivo Estratégico' : 'tema',
    'Secretaria Responsável' : 'orgao',
    'ODS vinculados' : 'ods',
    'Indicador' : 'indicador'
}

In [7]:
[k for k in df.keys() if k not in de_para_metas]

[]

In [8]:
df = df.rename(de_para_metas, axis=1)

In [9]:
df['pdm_id'] = 1

In [10]:
df['id'] = df.index + 1

In [11]:
def normalize_table(df, col, renamed_col = False):
    
    table = df[[col, 'pdm_id']]
    table = table.drop_duplicates().reset_index(drop=True)
    table['id'] = table.index+1
    
    mapped = table[['id', col]].to_dict(orient='dict')[col]
    #key+1 para coluna id começar em 1
    mapped = {val : key+1 for key, val in mapped.items()}
    
    df[col+'_id'] = df[col].apply(lambda x: mapped[x])
    df.drop(col, axis=1, inplace=True)
    
    if renamed_col:
        table.rename({col : renamed_col}, axis=1, inplace=True)
    
    
    return table

In [12]:
macrotemas = normalize_table(df, 'macrotema', 'descricao')

In [13]:
macrotemas

,descricao,pdm_id,id
0,SP Justa e Inclusiva,1,1
1,SP Segura e Bem Cuidada,1,2
2,SP Ágil,1,3
3,SP Inovadora e Criativa,1,4
4,SP Global e Sustentável,1,5
5,SP Eficiente,1,6


In [14]:
temas = normalize_table(df, 'tema', 'descricao')

In [15]:
temas.head()

,descricao,pdm_id,id
0,Reduzir a pobreza e ampliar o acesso a direito...,1,1
1,Garantir à população atendimento integral em s...,1,2
2,Garantir a proteção integral e o pleno desenvo...,1,3
3,"Promover o acesso à moradia, à urbanização e à...",1,4
4,Garantir à população idosa o pleno exercício d...,1,5


In [16]:
#não usamos subtema
df['subtema_id'] = None

In [17]:
def extrair_orgaos(df):
    
    orgaos = df['orgao'].unique()
    orgaos_unicos = set()
    
    for item in orgaos:
        splited = item.split(',')
        for sigla in splited:
            sigla = sigla.strip()
            orgaos_unicos.add(sigla)
            
    return orgaos_unicos

In [18]:
orgaos_unicos = extrair_orgaos(df)

In [19]:
orgaos_table = pd.read_csv(solve_path('orgaos.csv',
                                      solve_path('estrutura_administrativa_simplificada',
                                                'generated_data')),
                          sep=';', encoding='latin-1')

In [20]:
orgaos_table.sample(5)

,id,tipo_orgao_id,sigla,descricao
81,18,3,FPETC,"Fundação Paulistana de Educação, Tecnologia e ..."
73,10,3,SP NEGÓCIOS,São Paulo Negócios
29,30,1,SMTUR,Secretaria Municipal de Turismo
70,7,3,COHAB-SP,Companhia Metropolitana de Habitação de São Paulo
38,7,2,SUBCT,Subprefeitura Cidade Tiradentes


In [21]:
for sigla in orgaos_unicos:
    if sigla not in orgaos_table['sigla'].unique():
        print(sigla)
else:
    print('Todos os órgãos já estão na tabela')

Todos os órgãos já estão na tabela


In [22]:
def sigla_in_orgaos(sigla_alvo, orgao_cell):
    
    splited = orgao_cell.split(',')
    
    for sigla in splited:
        sigla = sigla.strip()
        if sigla==sigla_alvo:
            return True
    else:
        return False

def cross_table_orgaos(df, orgaos_table):
    
    mapped = orgaos_table.to_dict()['sigla']
    #key+1 para indexar em 1 não 0
    mapped = {val : key+1 for key, val in mapped.items()}
    
    data = []
    id_ = 1
    for i, row in df.iterrows():
        orgao_cell = row['orgao']
        meta_id = row['id']
        for orgao_sigla, orgao_id in mapped.items():
            if sigla_in_orgaos(orgao_sigla, orgao_cell):
                #colocando False para todos no orgao responsavel
                #depois equipe CP atualiza essa info
                data_row = (id_, meta_id, orgao_id, False)
                data.append(data_row)
                id_+=1
    df.drop('orgao', axis=1, inplace=True)
    
    return pd.DataFrame(data, columns = ('id', 'meta_id', 
                                         'orgao_id', 'orgao_responsavel'))

In [23]:
meta_orgao = cross_table_orgaos(df, orgaos_table)

In [24]:
meta_orgao.sample(5)

,id,meta_id,orgao_id,orgao_responsavel
62,63,41,27,False
73,74,51,17,False
14,15,10,20,False
13,14,10,14,False
78,79,55,3,False


In [25]:
def save_df(df, fname, folder = GENERATED_DATA_DIR):
    
    fname = solve_path(fname, folder)
    
    try:
        df.to_csv(fname, sep =';', decimal=',', 
                  encoding='latin-1', index=False)
    except UnicodeEncodeError:
        df.to_csv(fname, sep =';', decimal=',', 
                  encoding='utf-8', index=False)

In [26]:
df.rename({'indicador' : 'indicador_temp',
          'ods' : 'ods_temp'}, axis=1, inplace=True)

In [27]:
df['inativa'] = False
df['status'] = None
df['motivo_inativacao'] = None
df['data_inativacao'] = None

In [28]:
df.fillna(value='NULL', inplace=True)

In [29]:
df.keys()

Index(['codigo', 'titulo', 'indicador_temp', 'contexto', 'complemento',
       'ods_temp', 'pdm_id', 'id', 'macrotema_id', 'tema_id', 'subtema_id',
       'inativa', 'status', 'motivo_inativacao', 'data_inativacao'],
      dtype='object')

In [30]:
col_order = [
             'id', 
             'pdm_id',
             'codigo', 
             'titulo', 
             'contexto',
             'complemento',
             'macrotema_id', 
             'tema_id', 
             'subtema_id',
             'status', 
             'inativa', 
             'motivo_inativacao', 
             'data_inativacao',
             'indicador_temp',
             'ods_temp',
            ]

In [31]:
df = df[col_order]

In [32]:
save_df(df, 'Meta.csv')

In [33]:
save_df(macrotemas, 'macro_tema.csv')

In [34]:
save_df(temas, 'tema.csv')

In [35]:
save_df(meta_orgao, 'Meta_Orgao.csv')

In [36]:
#vou guardar o dataframe principal como "metas"
#para seguir com a analise das iniciativas
metas = df

In [37]:
iniciativas = download_xl(link_pdm_final, 'pdm_versao_final_iniciativas.xlsx',
                 ORIGINAL_DATA_DIR, skiprows=1, sheet_name='iniciativas')

In [38]:
iniciativas.head()

,Meta_numero,Meta_descricao,Iniciativa,Iniciativa_descricao
0,1,Atender 1.700.000 pessoas em programas de tran...,a,Atender 1.250.000 pessoas em programas de tran...
1,1,Atender 1.700.000 pessoas em programas de tran...,b,Atender 450.000 pessoas em programas de transf...
2,1,Atender 1.700.000 pessoas em programas de tran...,c,Aprimorar e racionalizar os cadastros dos bene...
3,2,Implantar o Prontuário Eletrônico em 100% das ...,a,Adquirir equipamentos para implantação do pron...
4,2,Implantar o Prontuário Eletrônico em 100% das ...,b,Implantar infraestrutura de tecnologia da info...


In [39]:
iniciativas['codigo'] = iniciativas.apply(lambda row: str(row['Meta_numero']) + '.' + row['Iniciativa'],
                                         axis=1)

In [40]:
iniciativas = iniciativas.rename({'Iniciativa_descricao' : 'titulo'}, axis=1)

In [41]:
iniciativas['descricao'] = None
iniciativas['compoe_indicador_meta'] = None
iniciativas['status'] = None

In [42]:
#checando se o id da meta é igual o código da meta
(metas['codigo']==metas['id']).all()

True

In [43]:
iniciativas = iniciativas.rename({'Meta_numero' : 'meta_id'}, axis=1)

In [44]:
iniciativas['id'] = iniciativas.index+1

In [45]:
cols = ['id', 'meta_id', 'codigo', 'titulo', 'descricao', 'compoe_indicador_meta', 'status']

In [46]:
iniciativas = iniciativas[cols]

In [47]:
iniciativas.head()

,id,meta_id,codigo,titulo,descricao,compoe_indicador_meta,status
0,1,1,1.a,Atender 1.250.000 pessoas em programas de tran...,None,None,None
1,2,1,1.b,Atender 450.000 pessoas em programas de transf...,None,None,None
2,3,1,1.c,Aprimorar e racionalizar os cadastros dos bene...,None,None,None
3,4,2,2.a,Adquirir equipamentos para implantação do pron...,None,None,None
4,5,2,2.b,Implantar infraestrutura de tecnologia da info...,None,None,None


In [48]:
iniciativas.fillna('NULL', inplace=True)

In [49]:
save_df(iniciativas, 'Iniciativa.csv')

In [50]:
indicadores = metas[['codigo', 'indicador_temp']]

In [51]:
indicadores = indicadores.rename({'codigo' : 'id_meta',
                                 'indicador_temp' : 'titulo'}, axis=1)

In [52]:
indicadores = indicadores.reset_index(drop=True)
indicadores['id'] = indicadores.index+1

In [53]:
df_origi = download_xl(link_pdm_final, 'pdm_versao_final.xlsx',
                 ORIGINAL_DATA_DIR, skiprows=1)

In [54]:
indicadores['codigo'] = 'indi_' + indicadores['id'].astype(str)

In [55]:
cols_indicadores = [
    'id',
    'id_meta',
    'id_iniciativa',
    'id_atividade',
    'codigo',
    'titulo',
    'contexto',
    'observacao',
    'polaridade',
    'tipo_regiao',
    'por_regiao',
    'agregador_id',
    'valor_base',
    'unidade',
    'periodicidade',
    'janela'
]

In [56]:
for col in cols_indicadores:
    if col not in indicadores.columns:
        indicadores[col] = None

In [57]:
indicadores.fillna('NULL', inplace=True)

In [58]:
indicadores = indicadores[cols_indicadores]

In [59]:
indicadores.head()

,id,id_meta,id_iniciativa,id_atividade,codigo,titulo,contexto,observacao,polaridade,tipo_regiao,por_regiao,agregador_id,valor_base,unidade,periodicidade,janela
0,1,1,NULL,NULL,indi_1,Média móvel dos últimos 12 meses do número de ...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1,2,2,NULL,NULL,indi_2,Percentual de Unidades Básicas de Saúde com pr...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2,3,3,NULL,NULL,indi_3,Número de equipamentos de saúde implantados (s...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
3,4,4,NULL,NULL,indi_4,Número de equipamentos de saúde reformados e/o...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4,5,5,NULL,NULL,indi_5,Número de centros implantados e em funcionamento.,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [60]:
save_df(iniciativas, 'Indicador.csv')

In [61]:
#PEGUEI OS ODS ABAIXO OD SITE DA ONU
ods_onu ='''ODS 1 – Erradicação da pobreza: acabar com a pobreza em todas as suas formas, em todos os lugares.;
ODS 2 – Fome zero e agricultura sustentável: acabar com a fome, alcançar a segurança alimentar e melhoria da nutrição e promover a agricultura sustentável.;
ODS 3 – Saúde e bem-estar: assegurar uma vida saudável e promover o bem-estar para todos, em todas as idades.;
ODS 4 – Educação de qualidade: assegurar a educação inclusiva, equitativa e de qualidade, e promover oportunidades de aprendizagem ao longo da vida para todos.;
ODS 5 – Igualdade de gênero: alcançar a igualdade de gênero e empoderar todas as mulheres e meninas.;
ODS 6 – Água potável e saneamento: garantir disponibilidade e manejo sustentável da água e saneamento para todos.;
ODS 7 – Energia limpa e acessível: garantir acesso à energia barata, confiável, sustentável e renovável para todos.;
ODS 8 – Trabalho decente e crescimento econômico: promover o crescimento econômico sustentado, inclusivo e sustentável, emprego pleno e produtivo, e trabalho decente para todos.;
ODS 9 – Indústria, inovação e infraestrutura: construir infraestrutura resiliente, promover a industrialização inclusiva e sustentável, e fomentar a inovação.;
ODS 10 – Redução das desigualdades: reduzir as desigualdades dentro dos países e entre eles.;
ODS 11 – Cidades e comunidades sustentáveis: tornar as cidades e os assentamentos humanos inclusivos, seguros, resilientes e sustentáveis.;
ODS 12 – Consumo e produção responsáveis: assegurar padrões de produção e de consumo sustentáveis.;
ODS 13 – Ação contra a mudança global do clima: tomar medidas urgentes para combater a mudança climática e seus impactos.;
ODS 14 – Vida na água: conservação e uso sustentável dos oceanos, dos mares e dos recursos marinhos para o desenvolvimento sustentável.;
ODS 15 – Vida terrestre: proteger, recuperar e promover o uso sustentável dos ecossistemas terrestres, gerir de forma sustentável as florestas, combater a desertificação, deter e reverter a degradação da Terra e deter a perda da biodiversidade.;
ODS 16 – Paz, justiça e instituições eficazes: promover sociedades pacíficas e inclusivas para o desenvolvimento sustentável, proporcionar o acesso à justiça para todos e construir instituições eficazes, responsáveis e inclusivas em todos os níveis.;
ODS 17 – Parcerias e meios de implementação: fortalecer os meios de implementação e revitalizar a parceria global para o desenvolvimento sustentável.'''

In [62]:
ods_onu = [ods.split(' – ') for ods in ods_onu.split(';')]

In [63]:
ods_onu = pd.DataFrame(data=ods_onu, columns=['numero', 'titulo'])

In [64]:
ods_onu['numero'] = ods_onu['numero'].str.strip('\n').str.replace('ODS', '').str.strip().astype(int)

In [65]:
ods_onu['descricao'] = ods_onu['titulo'].apply(lambda x: x.split(':')[1])
ods_onu['titulo'] = ods_onu['titulo'].apply(lambda x: x.split(':')[0])

In [66]:
ods_onu['id'] = ods_onu.index+1

In [67]:
(ods_onu['id']==ods_onu['numero']).all()

True

In [68]:
save_df(ods_onu, 'ods.csv')

In [69]:
link_ods = 'https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/governo/arquivos/Agenda%202030/vinculacao/vinculacao-pdm-agenda2030.xlsx'
ods_meta = download_xl(link_ods, 'relacao_metas_ods.xlsx', ORIGINAL_DATA_DIR, 
                  sheet_name='Vinculação PDM_ODS', skiprows=1)

In [70]:
#numero da meta esta com celula agregada
#se for nulo, tem que pegar valor anterior
ods_meta.head(5)

,Nº Meta PdM,Redação da Meta do Programa de Metas,ODS,Nº Meta ODS,Redação da Meta da Agenda Municipal 2030
0,1.0,Atender 1.700.000 pessoas em programas de tran...,1,Meta 1.3,"Assegurar, em nível municipal, até 2030, o ace..."
1,NaN,NaN,2,Meta 2.1,"Até 2030, reduzir a subnutrição crônica e agud..."
2,2.0,Implantar o Prontuário Eletrônico em 100% das ...,3,Meta 3.8,"Atingir a cobertura universal de saúde, o aces..."
3,3.0,Implantar 30 novos equipamentos de saúde no mu...,3,Meta 3.8,"Atingir a cobertura universal de saúde, o aces..."
4,4.0,Reformar e/ou reequipar 187 equipamentos de sa...,3,Meta 3.8,"Atingir a cobertura universal de saúde, o aces..."


In [71]:
#Nao vou fazer o mesmo para a redacao da meta
#porque nao vou usar essa coluna
col_num_meta = 'Nº Meta PdM'
for i, row in ods_meta.iterrows():
    
    if pd.isnull(row[col_num_meta]):
        num_meta = ods_meta.loc[i-1, col_num_meta]
        ods_meta.loc[i, col_num_meta] = num_meta

In [72]:
ods_meta.head()

,Nº Meta PdM,Redação da Meta do Programa de Metas,ODS,Nº Meta ODS,Redação da Meta da Agenda Municipal 2030
0,1.0,Atender 1.700.000 pessoas em programas de tran...,1,Meta 1.3,"Assegurar, em nível municipal, até 2030, o ace..."
1,1.0,NaN,2,Meta 2.1,"Até 2030, reduzir a subnutrição crônica e agud..."
2,2.0,Implantar o Prontuário Eletrônico em 100% das ...,3,Meta 3.8,"Atingir a cobertura universal de saúde, o aces..."
3,3.0,Implantar 30 novos equipamentos de saúde no mu...,3,Meta 3.8,"Atingir a cobertura universal de saúde, o aces..."
4,4.0,Reformar e/ou reequipar 187 equipamentos de sa...,3,Meta 3.8,"Atingir a cobertura universal de saúde, o aces..."


In [73]:
tags = ods_meta[['Nº Meta PdM','Nº Meta ODS','Redação da Meta da Agenda Municipal 2030', 'ODS']].copy()

In [74]:
tags['descricao'] = tags['Nº Meta ODS'] + ' : ' + tags['Redação da Meta da Agenda Municipal 2030']

In [75]:
tags.rename({'ODS' : 'ods_id'}, axis=1, inplace=True)

In [76]:
tags['pdm_id'] = 1

In [77]:
tags['icone']='NULL'

In [78]:
tags.head()

,Nº Meta PdM,Nº Meta ODS,Redação da Meta da Agenda Municipal 2030,ods_id,descricao,pdm_id,icone
0,1.0,Meta 1.3,"Assegurar, em nível municipal, até 2030, o ace...",1,"Meta 1.3 : Assegurar, em nível municipal, até ...",1,NULL
1,1.0,Meta 2.1,"Até 2030, reduzir a subnutrição crônica e agud...",2,"Meta 2.1 : Até 2030, reduzir a subnutrição crô...",1,NULL
2,2.0,Meta 3.8,"Atingir a cobertura universal de saúde, o aces...",3,Meta 3.8 : Atingir a cobertura universal de sa...,1,NULL
3,3.0,Meta 3.8,"Atingir a cobertura universal de saúde, o aces...",3,Meta 3.8 : Atingir a cobertura universal de sa...,1,NULL
4,4.0,Meta 3.8,"Atingir a cobertura universal de saúde, o aces...",3,Meta 3.8 : Atingir a cobertura universal de sa...,1,NULL


In [79]:
tags = tags.reset_index(drop=True)
tags['id'] = tags.index+1

In [80]:
meta_tag = tags[['id', 'Nº Meta PdM']]

In [81]:
tags = tags[['id', 'pdm_id', 'descricao', 'icone', 'ods_id']]

In [82]:
tags = tags.drop_duplicates(subset=['descricao', 'ods_id'])

In [83]:
tags.head()

,id,pdm_id,descricao,icone,ods_id
0,1,1,"Meta 1.3 : Assegurar, em nível municipal, até ...",NULL,1
1,2,1,"Meta 2.1 : Até 2030, reduzir a subnutrição crô...",NULL,2
2,3,1,Meta 3.8 : Atingir a cobertura universal de sa...,NULL,3
6,7,1,"Meta 3.2 : Até 2030, reduzir no município de S...",NULL,3
10,11,1,"Meta 4.2 : Até 2030, garantir que todas as men...",NULL,4


In [84]:
save_df(tags, 'tag.csv')

In [85]:
meta_tag = meta_tag.rename({'id' : 'tag_id',
                'Nº Meta PdM' : 'meta_id'}, axis=1)

In [86]:
meta_tag['meta_id'] = meta_tag['meta_id'].astype(int)

In [87]:
meta_tag = meta_tag.reset_index(drop=True)
meta_tag['id'] = meta_tag.index+1

In [88]:
meta_tag.head()

,tag_id,meta_id,id
0,1,1,1
1,2,1,2
2,3,2,3
3,4,3,4
4,5,4,5


In [89]:
save_df(meta_tag, 'Meta_tag.csv')